# Clean up the protein groups dataset for easier analysis

In [94]:
import pandas as pd
import numpy as np
import re
import urllib, json

## Import and filter patient data

In [119]:
data = pd.read_csv('proteinGroups_simplified.txt', sep='\t')
data = data.filter(regex = 'LFQ|Protein').rename(columns=lambda x: re.sub('LFQ intensity ','',x))

## Remove invalid rows (REV, CON, and rows without proper protein identifiers)

In [120]:
data = data[~data['Protein IDs'].str.contains('REV|CON')]
data = data[data['Protein IDs'].str.contains('[A-Z]')]
data

,Protein IDs,01OV007,01OV007_NM,01OV008_NM,01OV010_NM,01OV013_NM,01OV017,01OV017_NM,01OV018,01OV019_NM,01OV023,01OV023_NM,01OV024_NM,01OV026,01OV029,01OV029_NM,01OV030,01OV039,01OV039_NM,01OV041,02OV001_NM,02OV005,02OV006,02OV008,02OV015,02OV022,02OV023,02OV032,04OV001,04OV004,04OV005,04OV008,04OV011,04OV012,04OV013,04OV017,04OV018,04OV021,04OV023,04OV024,04OV027,04OV028,11OV002_NM,11OV010_NM,14OV011,14OV011_NM,14OV029_NM,15OV001,15OV001_NM,17OV001,17OV001_NM,17OV002,17OV002_NM,17OV003_NM,17OV004_NM,17OV005_NM,17OV014,17OV015,17OV018,17OV026
0,A0AV96;Q9NQ94,6.632300e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.547900e+08,0.000000e+00,5.532400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500300e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.421300e+07,0.000000e+00,8.745400e+07,0.000000e+00,0.000000e+00,1.628300e+08,1.949700e+08,1.923800e+08,7.927300e+07,9.002100e+07,1.070000e+08,1.278000e+08,9.114100e+07,0.000000e+00,2.317500e+08,7.014400e+07,0.000000e+00,6.846900e+07,1.024000e+08,1.078900e+08,1.591000e+08,1.228400e+08,9.584100e+07,8.291400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761400e+07,0.000000e+00,6.270100e+07,2.525300e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579100e+08,5.738500e+07
1,A0AVT1,1.191900e+09,0.000000e+00,4.044000e+08,0.000000e+00,0.000000e+00,1.436700e+09,1.148600e+09,2.047900e+09,1.343800e+09,2.071800e+09,8.754800e+08,1.248400e+08,1.151600e+09,2.272500e+09,6.599700e+08,7.827700e+08,3.885500e+09,1.051000e+09,8.061900e+08,7.565100e+08,1.430100e+09,1.807800e+09,1.292700e+09,1.247500e+09,1.880200e+09,1.077600e+09,2.856200e+09,2.005000e+09,1.032100e+09,1.058700e+09,1.318600e+09,8.865900e+08,1.715700e+09,1.782000e+09,2.464900e+09,1.220300e+09,1.560400e+09,2.007700e+09,1.005800e+09,1.472800e+09,1.625600e+09,5.856800e+08,3.971100e+08,1.649900e+09,2.624500e+08,6.132900e+08,1.821500e+09,7.370200e+08,2.196100e+09,9.756600e+08,2.103900e+09,0.000000e+00,2.257900e+08,0.000000e+00,0.000000e+00,1.092300e+09,1.640500e+09,1.831100e+09,1.045900e+09
2,A0FGR8,8.069400e+08,0.000000e+00,0.000000e+00,0.000000e+00,2.732400e+08,4.981100e+08,2.522700e+08,3.686300e+08,6.054700e+08,3.958100e+08,3.767000e+08,3.824600e+08,3.835000e+08,5.610600e+08,6.113000e+08,1.320800e+09,3.497000e+08,5.931000e+08,4.130700e+08,5.407700e+08,4.453900e+08,5.926400e+08,5.773900e+08,3.446400e+08,6.350400e+08,6.369200e+08,6.425800e+08,4.567600e+08,4.627600e+08,4.668000e+08,4.844100e+08,4.037200e+08,4.380600e+08,4.558900e+08,5.982100e+08,4.298900e+08,5.554100e+08,3.125100e+08,6.301900e+08,2.836900e+08,3.770000e+08,3.888800e+08,4.131800e+08,4.986200e+08,5.080000e+08,5.424400e+08,4.146700e+08,6.754500e+08,2.999000e+08,4.602200e+08,4.069700e+08,0.000000e+00,6.238700e+08,0.000000e+00,0.000000e+00,6.994500e+08,4.626500e+08,5.260800e+08,4.205100e+08
3,A0JLT2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.914400e+07,0.000000e+00
4,A0M8Q6,1.087100e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.187900e+08,0.000000e+00,0.000000e+00,0.000000e+00,1.289700e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.460400e+08,0.000000e+00,1.146900e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000

## Split up rows containing multiple protein identifiers

In [121]:
# Thanks StackOverflow! 
# https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

data = tidy_split(data, 'Protein IDs', sep=';')
data

,Protein IDs,01OV007,01OV007_NM,01OV008_NM,01OV010_NM,01OV013_NM,01OV017,01OV017_NM,01OV018,01OV019_NM,01OV023,01OV023_NM,01OV024_NM,01OV026,01OV029,01OV029_NM,01OV030,01OV039,01OV039_NM,01OV041,02OV001_NM,02OV005,02OV006,02OV008,02OV015,02OV022,02OV023,02OV032,04OV001,04OV004,04OV005,04OV008,04OV011,04OV012,04OV013,04OV017,04OV018,04OV021,04OV023,04OV024,04OV027,04OV028,11OV002_NM,11OV010_NM,14OV011,14OV011_NM,14OV029_NM,15OV001,15OV001_NM,17OV001,17OV001_NM,17OV002,17OV002_NM,17OV003_NM,17OV004_NM,17OV005_NM,17OV014,17OV015,17OV018,17OV026
0,A0AV96,6.632300e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.547900e+08,0.000000e+00,5.532400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500300e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.421300e+07,0.000000e+00,8.745400e+07,0.000000e+00,0.000000e+00,1.628300e+08,1.949700e+08,1.923800e+08,7.927300e+07,9.002100e+07,1.070000e+08,1.278000e+08,9.114100e+07,0.000000e+00,2.317500e+08,7.014400e+07,0.000000e+00,6.846900e+07,1.024000e+08,1.078900e+08,1.591000e+08,1.228400e+08,9.584100e+07,8.291400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761400e+07,0.000000e+00,6.270100e+07,2.525300e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579100e+08,5.738500e+07
0,Q9NQ94,6.632300e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.547900e+08,0.000000e+00,5.532400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500300e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.421300e+07,0.000000e+00,8.745400e+07,0.000000e+00,0.000000e+00,1.628300e+08,1.949700e+08,1.923800e+08,7.927300e+07,9.002100e+07,1.070000e+08,1.278000e+08,9.114100e+07,0.000000e+00,2.317500e+08,7.014400e+07,0.000000e+00,6.846900e+07,1.024000e+08,1.078900e+08,1.591000e+08,1.228400e+08,9.584100e+07,8.291400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761400e+07,0.000000e+00,6.270100e+07,2.525300e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579100e+08,5.738500e+07
1,A0AVT1,1.191900e+09,0.000000e+00,4.044000e+08,0.000000e+00,0.000000e+00,1.436700e+09,1.148600e+09,2.047900e+09,1.343800e+09,2.071800e+09,8.754800e+08,1.248400e+08,1.151600e+09,2.272500e+09,6.599700e+08,7.827700e+08,3.885500e+09,1.051000e+09,8.061900e+08,7.565100e+08,1.430100e+09,1.807800e+09,1.292700e+09,1.247500e+09,1.880200e+09,1.077600e+09,2.856200e+09,2.005000e+09,1.032100e+09,1.058700e+09,1.318600e+09,8.865900e+08,1.715700e+09,1.782000e+09,2.464900e+09,1.220300e+09,1.560400e+09,2.007700e+09,1.005800e+09,1.472800e+09,1.625600e+09,5.856800e+08,3.971100e+08,1.649900e+09,2.624500e+08,6.132900e+08,1.821500e+09,7.370200e+08,2.196100e+09,9.756600e+08,2.103900e+09,0.000000e+00,2.257900e+08,0.000000e+00,0.000000e+00,1.092300e+09,1.640500e+09,1.831100e+09,1.045900e+09
2,A0FGR8,8.069400e+08,0.000000e+00,0.000000e+00,0.000000e+00,2.732400e+08,4.981100e+08,2.522700e+08,3.686300e+08,6.054700e+08,3.958100e+08,3.767000e+08,3.824600e+08,3.835000e+08,5.610600e+08,6.113000e+08,1.320800e+09,3.497000e+08,5.931000e+08,4.130700e+08,5.407700e+08,4.453900e+08,5.926400e+08,5.773900e+08,3.446400e+08,6.350400e+08,6.369200e+08,6.425800e+08,4.567600e+08,4.627600e+08,4.668000e+08,4.844100e+08,4.037200e+08,4.380600e+08,4.558900e+08,5.982100e+08,4.298900e+08,5.554100e+08,3.125100e+08,6.301900e+08,2.836900e+08,3.770000e+08,3.888800e+08,4.131800e+08,4.986200e+08,5.080000e+08,5.424400e+08,4.146700e+08,6.754500e+08,2.999000e+08,4.602200e+08,4.069700e+08,0.000000e+00,6.238700e+08,0.000000e+00,0.000000e+00,6.994500e+08,4.626500e+08,5.260800e+08,4.205100e+08
3,A0JLT2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0

## Map UniProt IDs to gene names

### First we need to create groupings of IDs to send to the UniProt API. This allows us to do fewer total requests without requesting too many at once.

In [87]:
# Number of proteins in each group
n = 300
# Grab all the protein IDs as a big list
id_list = data['Protein IDs'].tolist()
# Split the list into a list of request group lists
grouped_list = [id_list[i * n:(i+1) * n] for i in range((len(id_list) + n-1) // n)]

### Now we query UniProt multiple times to create a big dictionary which maps all of our protein IDs to gene names
This is necessary because some of the protein IDs have been removed from UniProt and therefore will not receive a response, so we can't trust that our list of gene names will directly line up with the dataframe

In [89]:
url = 'https://www.uniprot.org/uploadlists/'
contact = 'aeoliphant@gmail.com'
geneNames = {}

for group in grouped_list:
    # Put list into the correct format and add to request
    group_str = ' '.join(group)
    params = {'from':'ID', 'to':'GENENAME','format': 'tab','query': group_str}

    # Send the request to the UniProt API
    reqData = urllib.parse.urlencode(params)
    reqData = reqData.encode('utf-8')
    req = urllib.request.Request(url, reqData)
    req.add_header('User-Agent', 'Python %s' % contact)
    resp = urllib.request.urlopen(req)

    # Divide bytestring response into a list and add each mapping to the dictionary
    # The response looks like this: ['From', 'To', 'A0AV96', 'RBM47', 'B3EWG6', 'FAM25G']
    respData = str.split(str(resp.read(), 'utf-8'))
    geneData = respData[2:]

    for i in range(0, len(geneData), 2):
        geneNames[geneData[i]] = geneData[i+1]

In [91]:
len(geneNames)

7034

## Add gene names to the dataframe and change the index
This is a non-exhaustive mapping; any gene names that are missing will be changed to NA since we can't use them

In [122]:
# Mapping
data['Protein IDs'] = data['Protein IDs'].map(geneNames).fillna(np.nan)
# Remove rows with no gene name
data = data[data['Protein IDs'].notnull()]
data.columns = data.columns.str.replace('Protein IDs', 'Gene_Name')
data = data.set_index('Gene_Name')
data

,01OV007,01OV007_NM,01OV008_NM,01OV010_NM,01OV013_NM,01OV017,01OV017_NM,01OV018,01OV019_NM,01OV023,01OV023_NM,01OV024_NM,01OV026,01OV029,01OV029_NM,01OV030,01OV039,01OV039_NM,01OV041,02OV001_NM,02OV005,02OV006,02OV008,02OV015,02OV022,02OV023,02OV032,04OV001,04OV004,04OV005,04OV008,04OV011,04OV012,04OV013,04OV017,04OV018,04OV021,04OV023,04OV024,04OV027,04OV028,11OV002_NM,11OV010_NM,14OV011,14OV011_NM,14OV029_NM,15OV001,15OV001_NM,17OV001,17OV001_NM,17OV002,17OV002_NM,17OV003_NM,17OV004_NM,17OV005_NM,17OV014,17OV015,17OV018,17OV026
Gene_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
RBM47,6.632300e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.547900e+08,0.000000e+00,5.532400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500300e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.421300e+07,0.000000e+00,8.745400e+07,0.000000e+00,0.000000e+00,1.628300e+08,1.949700e+08,1.923800e+08,7.927300e+07,9.002100e+07,1.070000e+08,1.278000e+08,9.114100e+07,0.000000e+00,2.317500e+08,7.014400e+07,0.000000e+00,6.846900e+07,1.024000e+08,1.078900e+08,1.591000e+08,1.228400e+08,9.584100e+07,8.291400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761400e+07,0.000000e+00,6.270100e+07,2.525300e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579100e+08,5.738500e+07
A1CF,6.632300e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.547900e+08,0.000000e+00,5.532400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500300e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.421300e+07,0.000000e+00,8.745400e+07,0.000000e+00,0.000000e+00,1.628300e+08,1.949700e+08,1.923800e+08,7.927300e+07,9.002100e+07,1.070000e+08,1.278000e+08,9.114100e+07,0.000000e+00,2.317500e+08,7.014400e+07,0.000000e+00,6.846900e+07,1.024000e+08,1.078900e+08,1.591000e+08,1.228400e+08,9.584100e+07,8.291400e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761400e+07,0.000000e+00,6.270100e+07,2.525300e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579100e+08,5.738500e+07
UBA6,1.191900e+09,0.000000e+00,4.044000e+08,0.000000e+00,0.000000e+00,1.436700e+09,1.148600e+09,2.047900e+09,1.343800e+09,2.071800e+09,8.754800e+08,1.248400e+08,1.151600e+09,2.272500e+09,6.599700e+08,7.827700e+08,3.885500e+09,1.051000e+09,8.061900e+08,7.565100e+08,1.430100e+09,1.807800e+09,1.292700e+09,1.247500e+09,1.880200e+09,1.077600e+09,2.856200e+09,2.005000e+09,1.032100e+09,1.058700e+09,1.318600e+09,8.865900e+08,1.715700e+09,1.782000e+09,2.464900e+09,1.220300e+09,1.560400e+09,2.007700e+09,1.005800e+09,1.472800e+09,1.625600e+09,5.856800e+08,3.971100e+08,1.649900e+09,2.624500e+08,6.132900e+08,1.821500e+09,7.370200e+08,2.196100e+09,9.756600e+08,2.103900e+09,0.000000e+00,2.257900e+08,0.000000e+00,0.000000e+00,1.092300e+09,1.640500e+09,1.831100e+09,1.045900e+09
ESYT2,8.069400e+08,0.000000e+00,0.000000e+00,0.000000e+00,2.732400e+08,4.981100e+08,2.522700e+08,3.686300e+08,6.054700e+08,3.958100e+08,3.767000e+08,3.824600e+08,3.835000e+08,5.610600e+08,6.113000e+08,1.320800e+09,3.497000e+08,5.931000e+08,4.130700e+08,5.407700e+08,4.453900e+08,5.926400e+08,5.773900e+08,3.446400e+08,6.350400e+08,6.369200e+08,6.425800e+08,4.567600e+08,4.627600e+08,4.668000e+08,4.844100e+08,4.037200e+08,4.380600e+08,4.558900e+08,5.982100e+08,4.298900e+08,5.554100e+08,3.125100e+08,6.301900e+08,2.836900e+08,3.770000e+08,3.888800e+08,4.131800e+08,4.986200e+08,5.080000e+08,5.424400e+08,4.146700e+08,6.754500e+08,2.999000e+08,4.602200e+08,4.069700e+08,0.000000e+00,6.238700e+08,0.000000e+00,0.000000e+00,6.994500e+08,4.626500e+08,5.260800e+08,4.205100e+08
MED19,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,

## Save dataframe as a new file

In [125]:
cleaned_data = open('proteinGroups_cleaned.txt', 'w')
data.to_csv(cleaned_data, sep = '\t')
cleaned_data.close()